In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv('train.csv.zip')

In [3]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
embedding_index={}

In [10]:
glove_file = open('glove.6B.300d.txt',encoding="utf8")

In [7]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-04-19 05:11:00--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-19 05:11:00--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-19 05:11:00--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [8]:
# !unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [11]:
for line in glove_file:
    values = line.split()
    embedding_index[values[0]] = np.asarray(values[1:],dtype='float32')
glove_file.close()    

In [12]:
data.shape

(404290, 6)

In [13]:
data=data.dropna()
data.shape

(404287, 6)

In [14]:
qstin1_list = list(data['question1'])
qstin2_list = list(data['question2'])
texts = qstin1_list+qstin2_list

In [15]:
len(embedding_index)

400000

In [16]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index

In [17]:
embedding_dimension=100

In [18]:
embedding_matrix = np.zeros((len(word_index)+1,embedding_dimension))

In [19]:
len(word_index)

95595

In [20]:
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector[:embedding_dimension]

In [21]:
embedding_matrix.shape

(95596, 100)

In [22]:
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

In [23]:
embedding_layer = Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights = [embedding_matrix],input_length = 100)

In [47]:
X_ques1 = tokenizer.texts_to_sequences(qstin1_list)
X_ques1 = pad_sequences(X_ques1, maxlen=30,padding='post')

In [48]:
X_ques2 = tokenizer.texts_to_sequences(qstin2_list)
X_ques2 = pad_sequences(X_ques2,maxlen = 30,padding='post')

In [49]:
data['X_ques1'] = list(X_ques1)
data['X_ques2'] = list(X_ques2)

In [50]:
print(X_ques1[0])

[   2    3    1 1222   57 1222 2581    7  576    8  763  383    8   35
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [51]:
data.head(8)

,id,qid1,qid2,question1,question2,is_duplicate,X_ques1,X_ques2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[2, 3, 1, 1222, 57, 1222, 2581, 7, 576, 8, 763...","[2, 3, 1, 1222, 57, 1222, 2581, 7, 576, 8, 763..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[2, 3, 1, 559, 10, 14300, 13598, 5, 4565, 0, 0...","[2, 43, 182, 25, 1, 82, 237, 11296, 1, 14300, ..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[4, 13, 5, 217, 1, 440, 10, 17, 361, 1827, 200...","[4, 13, 361, 440, 24, 3338, 57, 1344, 219, 109..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[16, 72, 5, 2774, 312, 2757, 4, 13, 5, 649, 19...","[87, 1, 4170, 37, 230, 2234, 1343, 230, 3, 245..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[23, 49, 7131, 8, 231, 1891, 2047, 10570, 12, ...","[23, 1945, 43, 1242, 8, 2047, 231, 0, 0, 0, 0,..."
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,"[2371, 5, 72, 6, 9925, 940, 4451, 813, 12, 445...","[144, 6, 4894, 9925, 940, 813, 12, 9297, 8, 99..."
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,"[31, 5, 126, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2355, 1530, 12, 1068, 33, 122, 12, 276, 46..."
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,"[4, 13, 5, 24, 6, 42, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 31, 5, 9, 7, 24, 6, 338, 0, 0, 0, 0, 0, 0,..."


In [52]:
labels = np.asarray(data['is_duplicate'])

In [53]:
len(labels)

404287

In [54]:
#functional API
from keras.models import Sequential

from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPool1D, Dense, Flatten, Dropout, Conv2D
from keras.layers import Bidirectional
import keras.backend as K
from keras.layers import Layer

In [55]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [56]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128,5,activation='relu'))
model.add(MaxPool1D(5))
model.add(Conv1D(128,5,activation='relu'))
model.add(MaxPool1D(5))
model.add(Dropout(0.25))
model.add(Bidirectional(LSTM(100,activation='tanh',dropout=0.2,recurrent_dropout=0.2,return_sequences=True)))
#model.add(attention())
model.add(Flatten())
model.add(Dense(36))
model.add(Dense(1,activation='softmax'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          9559600   
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 96, 128)           64128     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 19, 128)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 15, 128)           82048     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 3, 128)            0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 3, 128)            0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 3, 200)           

In [65]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(20,(3),activation='relu',input_shape=(30,30)))
model.add(MaxPool1D(5))
model.add(Conv1D(10,(5),activation='relu'))
model.add(MaxPool1D(5))
model.add(Dropout(0.25))
model.add(Bidirectional(LSTM(10,activation='tanh',dropout=0.2,recurrent_dropout=0.2,return_sequences=True)))
model.add(Bidirectional(LSTM(10,activation='tanh',dropout=0.2,recurrent_dropout=0.2)))
model.add(attention())
model.add(Flatten())
model.add(Dense(5))
model.add(Dense(1,activation='softmax'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          9559600   
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 98, 20)            6020      
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 19, 20)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 15, 10)            1010      
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 3, 10)             0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 3, 10)             0         
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 3, 20)           

In [ ]:
# model = Sequential()
# model.add(embedding_layer)
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# model.summary()

In [38]:
train_len = int(len(X_ques1)*0.2)

In [39]:
train_len

80857

In [66]:
model.fit([X_ques1[0:train_len],X_ques2[0:train_len]], y=labels[0:train_len],batch_size=200,epochs=1, shuffle=True)

ValueError: ignored

In [ ]:
model.save('sample.h5')

In [41]:
model.evaluate([X_ques1[train_len:],X_ques2[train_len:]],labels[train_len:])

10108/10108 [==============================] - 75s 7ms/step - loss: 0.5520 - acc: 0.3683


[0.5520491003990173, 0.36827751994132996]

In [ ]:
dup_data = data[data['is_duplicate']==1]

In [ ]:
X_dup_ques1 = dup_data['X_ques1']
X_dup_ques2 = dup_data['X_ques2']

In [ ]:
dup_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,X_ques1,X_ques2
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,"[2371, 5, 72, 6, 9925, 940, 4451, 813, 12, 445...","[144, 6, 4894, 9925, 940, 813, 12, 9297, 8, 99..."
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,"[4, 13, 5, 24, 6, 42, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 31, 5, 9, 7, 24, 6, 338, 0, 0, 0, 0, 0, 0,..."
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1,"[4, 9, 5, 223, 12, 87, 17, 286, 1727, 0, 0, 0,...","[4, 13, 5, 165, 81, 17, 286, 1727, 0, 0, 0, 0,..."
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1,"[2, 13, 52, 609, 632, 7, 71, 0, 0, 0, 0, 0, 0,...","[4, 13, 15, 52, 609, 632, 7, 71, 0, 0, 0, 0, 0..."
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1,"[2, 59, 34, 94, 1403, 299, 39, 0, 0, 0, 0, 0, ...","[2, 59, 34, 94, 1403, 299, 0, 0, 0, 0, 0, 0, 0..."


In [ ]:
X_dup_ques1[5]

array([2371,    5,   72,    6, 9925,  940, 4451,  813,   12, 4451, 5037,
          2,   21,   28,  238,   46,   60,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

In [ ]:
model.predict([X_dup_ques1[0:5],X_dup_ques2[0:5]])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
help(model.evaluate)

Help on method evaluate in module tensorflow.python.keras.engine.training:

evaluate(x=None, y=None, batch_size=None, verbose=1, sample_weight=None, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, return_dict=False) method of tensorflow.python.keras.engine.sequential.Sequential instance
    Returns the loss value & metrics values for the model in test mode.
    
    Computation is done in batches (see the `batch_size` arg.)
    
    Arguments:
        x: Input data. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a list of tensors
            (in case the model has multiple inputs).
          - A dict mapping input names to the corresponding array/tensors,
            if the model has named inputs.
          - A `tf.data` dataset. Should return a tuple
            of either `(inputs, targets)` or
            `(inputs, targets, sample_

In [ ]:
(X_ques1[0:1])

array([[   2,    3,    1, 1222,   57, 1222, 2581,    7,  576,    8,  763,
         383,    8,   35,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]])

In [ ]:
X_ques1

array([[ 2,  3,  1, ...,  0,  0,  0],
       [ 2,  3,  1, ...,  0,  0,  0],
       [ 4, 13,  5, ...,  0,  0,  0],
       ...,
       [ 2,  3, 49, ...,  0,  0,  0],
       [ 2,  3,  1, ...,  0,  0,  0],
       [ 2,  3, 39, ...,  0,  0,  0]])

In [ ]:
X_ques1[0:train_len]

array([[  2,   3,   1, ...,   0,   0,   0],
       [  2,   3,   1, ...,   0,   0,   0],
       [  4,  13,   5, ...,   0,   0,   0],
       ...,
       [  4,  36, 159, ...,   0,   0,   0],
       [  3,   1, 244, ...,   0,   0,   0],
       [  2, 433,  10, ...,   0,   0,   0]])